In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel(f'./file/1.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：1


['https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=Door%2520Latch%2520Cable']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

list_url = []
while len(list_url) < input_.loc[a, 'SKU Count']:
    for page in range(input_.loc[a, 'Page']):
        b = 0
        while True:
            b += 1
            print(b, page, len(list_url))
            try:
                request_url = f'''{input_.loc[a, 'Url']}&start={page*100}&num=100'''
                
                resp = requests.get(request_url,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10))
                
                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    list_href = html.xpath('//h2[@class="item-headline"]/a/@href')
                    if list_href:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        list_url += [f'https://www.dormanproducts.com/{href.strip()}' for href in list_href]
        list_url = sorted(list(set(list_url)), key=list_url.index)

        # = = = = = = = = = = = = = = =

        if len(list_url) == input_.loc[a, 'SKU Count']:
            break

resp

1 0 0
2 0 0
1 1 100


<Response [200]>

In [6]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://www.dormanproducts.com/p-84706-924-351...
1,2,https://www.dormanproducts.com/p-65564-924-351...
2,3,https://www.dormanproducts.com/p-65557-924-352...
3,4,https://www.dormanproducts.com/p-129108-940-10...
4,5,https://www.dormanproducts.com/p-104645-912-60...
...,...,...
127,128,https://www.dormanproducts.com/p-125851-700-52...
128,129,https://www.dormanproducts.com/p-100837-961-30...
129,130,https://www.dormanproducts.com/p-73244-961-304...
130,131,https://www.dormanproducts.com/p-100761-963-20...


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_url_error.xlsx', index=False)

output_error

""
